In [1]:
import sys
c_path = "/home/fforge/Stage-IA3D/scripts/"
sys.path.append(f"{c_path}/mutations")
sys.path.append(f"{c_path}/orca_predictions")
sys.path.append(f"{c_path}/orcanalyse")

import mutate_module as mm
import process_sequence as ps
import matrices as mat

import pandas as pd
import os

# 

In [2]:
mutationfile = None
bedfile = "./resources/annotations/chr1_4_specific_ctcf_Bos.bed"
ref_fasta = "./resources/genome/Bos_taurus_chr1.fa"
mut_path = "./outputs/genome/mut_4_specific_CTCF_start_chr1_Bos_tau"
mutationtype = "shuffle"
nb_random = 1

# 

In [ ]:
chrom = "chr1"
prediction_prefix = "mut_4_specific_CTCF_chr1_first_32Mb_Bos_tau"
resol_model = "32Mb"
mpos = 16_000_000
cool_resol = 128_000
strict=False,
padding_chr=None,
use_cuda=False,
use_memmapgenome=False
pred_path = "./outputs/predictions/Bos_taurus_chr1"
orcarun_path = ".outputs/runfiles/start_chr1_Bos/"

# 

In [4]:
mm.main(mutationfile=None, 
        bed=bedfile, 
        genome=ref_fasta, 
        path=mut_path,
        mutationtype=mutationtype,
        nb_random=nb_random)

Valid mutated chrom chr1
Valid mutated chrom chr1


# 

In [ ]:
data = []
repository = ["Wtd_mut"] + [f"Rdm_mut_{i}" for i in range (nb_random)]

for name in repository:
    ps.main(chrom=chrom,
            output_prefix=f"{prediction_prefix}_{name}",
            mutation=name,
            resol_model=resol_model,
            mpos=mpos,
            fasta=f"{mut_path}/{name}/sequence.fa",
            cool_resol=cool_resol,
            strict=strict,
            padding_chr=padding_chr,
            use_cuda=False,
            use_memmapgenome=use_memmapgenome,
            pred_path=pred_path)
    
    path = f"{pred_path}/{prediction_prefix}_{name}"
    
    l_resol = mat.extract_resol_asc(path, "pred")

    data.append([f"orcarun_{name}", 
                 f"{l_resol}", 
                 f"{pred_path}/{prediction_prefix}_{name}", 
                 f"{name}", 
                 f"{ref_fasta.split('/')[-1].split('.')[0]}", 
                 f"OrcaRun"])

head = ["name", "list_resol", "path", "gtype", "genome", "mtype"]

print (os.path.exists(orcarun_path))
if orcarun_path and not os.path.exists(orcarun_path):
        print(orcarun_path)
        os.makedirs(orcarun_path)
        print (os.path.exists(orcarun_path))

df = pd.DataFrame(data,columns=head)
df.to_csv(f"{orcarun_path}/orcarun.csv", sep="\t", index=False, header=True)

ps.main(chrom=chrom,
        output_prefix="ref_orcarun",
        mutation="wt",
        resol_model=resol_model,
        mpos=mpos,
        fasta=ref_fasta,
        cool_resol=cool_resol,
        strict=strict,
        padding_chr=padding_chr,
        use_cuda=False,
        use_memmapgenome=use_memmapgenome,
        pred_path=pred_path)

ref_path = f"{pred_path}/ref_orcarun"
l_resol = mat.extract_resol_asc(ref_path, "pred")

data_ref = [f"orcarun_ref", 
            f"{l_resol}", 
            f"{pred_path}/ref_orcarun", 
            f"wt", 
            f"{ref_fasta.split('/')[-1].split('.')[0]}", 
            f"OrcaRun"]

df_ref = pd.DataFrame([data_ref], columns=head)
df_ref.to_csv(f"{orcarun_path}/ref_orcarun.csv", sep="\t", index=False, header=True, mode='w')


2025-04-15 11:42:59,139 - INFO - Time taken to load genome: 0.00 seconds
2025-04-15 11:43:02,394 - INFO - Time taken to get encoded sequence: 3.25 seconds
2025-04-15 11:43:07,070 - INFO - Time taken to load the ressources: 4.68 seconds
/home/fforge/miniforge3/envs/orca_env/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
2025-04-15 11:47:25,716 - INFO - Time taken for predictions: 258.64 seconds
2025-04-15 11:47:26,292 - INFO - Total time taken by main: 267.15 seconds
2025-04-15 11:47:26,308 - INFO - Time taken to load genome: 0.00 seconds
2025-04-15 11:47:29,407 - INFO - Time take

True


2025-04-15 11:51:55,135 - INFO - Time taken to get encoded sequence: 3.14 seconds
2025-04-15 11:51:59,626 - INFO - Time taken to load the ressources: 4.49 seconds
/home/fforge/miniforge3/envs/orca_env/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


# 

In [6]:
mat_comparisons = mat.build_CompareMatrices(filepathref=f"{orcarun_path}/ref_orcarun.csv", filepathcomp=f"{orcarun_path}/orcarun.csv")

TypeError: build_OrcaRun() got an unexpected keyword argument 'base_name'

# 

In [ ]:
mat_comparisons.heatmaps()

# 

In [ ]:
mat_comparisons.all_graphs()

# 

In [ ]:
mat_comparisons.correl_mat()

# 

In [ ]:
mat_comparisons.scores_regression()

In [ ]:
mat_comparisons.scores_regression(score_type="PC1")

In [ ]:
mat_comparisons.scores_regression(score_type="insulation_correl")